In [23]:
import gym
from myrl.environments import Envs
from myrl.policies import GaussianPolicy
from myrl.value_functions import ValueFunctionMLP, polyak
from myrl.visualizer import showit
from myrl.utils import ExperimentWriter, check_output, global_gradient_clip, list_gae, gae
from myrl.buffers import ReplayBuffer
from myrl.exploration import RandomNetworkBonus
import torch
import copy

envname = 'LunarLanderContinuous-v2'
# envname = 'Pendulum-v0'

wll = ExperimentWriter('tb/ppo_lun_GAE_128_evenout___')
env = gym.make(envname)
idim = env.observation_space.shape[0]
odim = env.action_space.shape[0]
envs = Envs(envname, 128)
envs.evenout(200)
idim, odim


126 / 128127 / 128

(8, 2)

In [21]:
env2 = gym.make('CartPole-v0')
env2.observation_space.sample()
wll = ExperimentWriter('tb/ppo_lun------------')


In [15]:
pi = GaussianPolicy([idim, 32, odim], std_start=1.1, std_min=0.1)
vfunc = ValueFunctionMLP([idim, 256, 128, 1])
tvfunc = copy.deepcopy(vfunc)

In [16]:
bonus = RandomNetworkBonus([idim, 128, 1], [idim, 128, 1], lr=1e-4, opt_steps=1)

In [17]:
opt = torch.optim.Adam(pi.parameters(), lr=3e-4)
copt = torch.optim.Adam(vfunc.parameters(), lr=1e-1)

In [20]:
def add_list2list(l1, l):
    for i in range(len(l)):
        l[i].append(l1[i])

pi.std = 0.9

In [22]:
wll.new()
writer = wll.writer
eps = 10000
gamma = 1
lmbda = 0.85

for ep in range(0, eps):
    l = envs.list_rollout(pi.act, gamma=gamma, length=10)
    rss = l[0][2].mean().item()
    adv = list_gae(l, tvfunc, gamma, lmbda=lmbda)
    for i in range(len(l)):
        # print(adv[i].shape, l[i][0].shape)
        pass
    add_list2list(adv, l)
    rbuff = ReplayBuffer(nitems=10, max_len=100000)
    rbuff.list_add(l)

    for imini_batch in range(60):
        oldobs, a, r, obs, d, oldprobs, smpls, muold, stdold, gae = rbuff.get(len(rbuff)//6)
        # adv = r + vfunc(obs).detach() - vfunc(oldobs)
        # print(gae.shape, oldobs.shape, "a"*5)
        # print(gae[:10], vfunc(oldobs)[:10])
        adv = gae.detach() - vfunc(oldobs)
        # print("--"*10)
        # print(adv[:4], gae[:4], vfunc(oldobs)[:4])
        adv = ((adv-adv.mean())/(adv.std()+1e-8)).detach()
        # print(adv[:10])
        # print("/"*10)
        for optstep in range(2):
            _, (newprobs, _, mu, std) = pi.act(oldobs, smpl=smpls)
            p = torch.distributions.Normal(muold, stdold)
            q = torch.distributions.Normal(mu, std)
            kldiv = torch.distributions.kl_divergence(p, q)
            if kldiv.mean().item() > 0.03:
                print("stopppp", kldiv.mean().item(), optstep)
                break
            # print(kldiv)
            ratio = torch.exp(newprobs-oldprobs)
            e = 0.15
            clipped = (ratio != torch.clamp(ratio, 1-e, 1+e)).float().mean().detach().item()
            loss = -torch.min(ratio*adv, torch.clamp(ratio, 1-e, 1+e)*adv).mean()
            opt.zero_grad()
            loss.backward()
            global_gradient_clip(pi)
            opt.step()
        
        for optstep in range(4):                    
            td = ((gae.detach() - vfunc(oldobs))**2).mean()
            # td = ((r + tvfunc(obs).detach() - vfunc(oldobs))**2).mean()
            copt.zero_grad()
            td.backward()
            global_gradient_clip(vfunc)
            copt.step()

    # print(vfunc(obs)[:5])

    polyak(vfunc, tvfunc, 1-1/40)


    if ep%1==0:
        print(ep, loss.item(), "r", rss, " td", td.item(), "clip", clipped, "___", pi.last_std.mean().item())
    if ep%20==0:
        print("showit=",showit(env, pi.act, max_steps=650))
        env.close()
        # print(mu.squeeze(1)[:20])


    writer.add_scalar("a/loss", loss.item(), ep)
    writer.add_scalar("a/td", td.item(), ep)
    writer.add_scalar("a/reward", r.mean(), ep)
    writer.add_scalar("a/clipped", clipped, ep)
    # writer.add_scalar("a/lr", opt.param_groups[0]['lr'], ep)



/home/darijan/mujoco-py/myrl/policies.py:34: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  h = torch.tensor(x, dtype=torch.float)
/home/darijan/.local/lib/python3.8/site-packages/torch/nn/functional.py:1558: UserWarning: nn.functional.tanh is deprecated. Use torch.tanh instead.
  warnings.warn("nn.functional.tanh is deprecated. Use torch.tanh instead.")


AssertionError: IsLocked() == false

In [80]:
import gym
import time
t1 = time.time()
from stable_baselines3 import PPO
from stable_baselines3.ppo import MlpPolicy
from stable_baselines3.common.cmd_util import make_vec_env0.3876180648803711

# Parallel environments
envname = 'Pendulum-v0'
env = make_vec_env(envname, n_envs=4)

model = PPO(MlpPolicy, env, verbose=2)
model.learn(total_timesteps=250000, log_interval=1, tb_log_name='/home/darijan/mujoco-py/tb/ppo_stable_4')

t2 = time.time()
print("total time = ", t2-t1)
env = gym.make(envname)
obs = env.reset()

for i in range(200):
    action, _states = model.predict(obs)
    obs, rewards, dones, info = env.step(action)
    env.render()
env.close()

           | 2.15e+03       |
|    n_updates            | 90             |
|    policy_gradient_loss | -0.000264      |
|    std                  | 0.919          |
|    value_loss           | 4.21e+03       |
--------------------------------------------
------------------------------------------
| rollout/                |              |
|    ep_len_mean          | 200          |
|    ep_rew_mean          | -1.17e+03    |
| time/                   |              |
|    fps                  | 609          |
|    iterations           | 11           |
|    time_elapsed         | 147          |
|    total_timesteps      | 90112        |
| train/                  |              |
|    approx_kl            | 0.0028223046 |
|    clip_fraction        | 0.0469       |
|    clip_range           | 0.2          |
|    entropy_loss         | -1.33        |
|    explained_variance   | -45.5        |
|    learning_rate        | 0.0003       |
|    loss                 | 2.46e+03     |
|    n_updates

In [36]:
import gym
import time
t1 = time.time()
from stable_baselines3 import PPO
from stable_baselines3.ppo import MlpPolicy
from stable_baselines3.common.cmd_util import make_vec_env

# Parallel environments
envname = 'LunarLanderContinuous-v2'
env = make_vec_env(envname, n_envs=4)

model = PPO(MlpPolicy, env, verbose=2)
# model.load('lunar_stable.sb')

model.learn(total_timesteps=250000*4, log_interval=1, tb_log_name='/home/darijan/mujoco-py/tb/ppo_stable_4')

t2 = time.time()
print("total time = ", t2-t1)
env = gym.make(envname)
obs = env.reset()


for i in range(200):
    action, _states = model.predict(obs)
    obs, rewards, dones, info = env.step(action)
    env.render()
    time.sleep(0.01)
env.close()

Using cuda device


RuntimeError: CUDA error: unspecified launch failure

In [ ]:
4000*256

In [30]:
model.save('lunar_stable.sb')

RuntimeError: cuda runtime error (719) : unspecified launch failure at /pytorch/torch/csrc/generic/serialization.cpp:26

In [31]:
env = gym.make(envname)
obs = env.reset()
rews = 0
for i in range(1000):
    action, _states = model.predict(obs)
    obs, rewards, dones, info = env.step(action)
    rews += rewards
    env.render()
    if dones:
        print(i, "last")
        break
    time.sleep(0.005)
env.close()
rews

RuntimeError: CUDA error: unspecified launch failure

In [20]:
model = model.load('lunar_stable.sb')


In [18]:
model = PPO(MlpPolicy, env, verbose=2)


Using cuda device
Wrapping the env in a DummyVecEnv.


In [52]:
# rs = torch.range(0, 5).unsqueeze(-1)
# rs = torch.ones(1000, 1)

def gaef(rs, obs, done, vfunc, gamma, lmbda):
    # print(rs.shape, obs.shape)
    vals = vfunc(obs).detach()
    T = len(vals)
    lmbda_pow = [lmbda**i for i in range(T)]
    rts = torch.zeros_like(rs)
    flag = torch.zeros_like(rts)
    def Rt(t):
        # print(rts.shape, obs.shape, vals.shape)

        if t == T-1:
            toret = rs[-1] + gamma*(1-done[-1])*vals[-1]
        else:
            toret = (rs[t]+  (1-lmbda)*gamma*vals[t]) + gamma*lmbda*Rt(t+1)
        rts[t] = toret
        flag[t] += 1
        return toret
    Rt(0)
    # print(rts.shape)
    return rts

def list_gae(list_all, vfunc, gamma, lmbda, rew_dim=2, obs_dim=3, done_dim=4):
    toret = []
    for i in range(len(list_all)):
        obs = list_all[i][obs_dim]
        done = list_all[i][done_dim]
        toret.append(gaef(list_all[i][rew_dim], obs, done, vfunc, gamma, lmbda))
    return toret

# gamma = 0.9
# lmbda = 1
# # vfunc = lambda obs: rs/(1-gamma)
# gaef(rs, rs, vfunc, gamma, lmbda)

In [242]:
def n_step_rewards_batch(r, gamma):
    zer = torch.zeros_like(r)
    discr = discounted_sum(r, gamma)
    rpadded = torch.cat((discr, zer), dim=1)
    minus_discounted = None

    for i in range(1, r.shape[1]):
        if minus_discounted is not None:
            minus_discounted = torch.cat(( minus_discounted, rpadded[:, i:i+r.shape[1]]), dim=-1)
        else:
            minus_discounted = rpadded[:, i:i+r.shape[1]]
    minus_discounted = torch.cat((minus_discounted, torch.zeros_like(discr)), dim=-1)
    return discr-minus_discounted

In [50]:
rs = torch.range(0, 9).unsqueeze(-1)
obs = rs
vfunctest = lambda obs: nes #torch.zeros(obs.shape[0], 1)+1
gae(rs, obs, vfunctest, 1, 0)

tensor([[45.],
        [46.],
        [46.],
        [45.],
        [43.],
        [40.],
        [36.],
        [31.],
        [25.],
        [ 9.]])

In [44]:
nes = rs.__reversed__().cumsum(dim=0).__reversed__()
nes

tensor([[45.],
        [45.],
        [44.],
        [42.],
        [39.],
        [35.],
        [30.],
        [24.],
        [17.],
        [ 9.]])

In [43]:
rs

tensor([[0.],
        [1.],
        [2.],
        [3.],
        [4.],
        [5.],
        [6.],
        [7.],
        [8.],
        [9.]])

In [46]:
def gae(rs, obs, vfunc, gamma, lmbda):
    vals = vfunc(obs).detach()
    T = len(vals)
    lmbda_pow = [lmbda**i for i in range(T)]
    rts = torch.zeros_like(rs)
    flag = torch.zeros_like(rts)
    def Rt(t):
        if t == T-1:
            toret = rs[-1]
        else:
            toret = rs[t]+ (1-lmbda)*gamma*vals[t] + gamma*lmbda*Rt(t+1)
        rts[t, 0] = toret
        flag[t] += 1
        return toret
    Rt(0)
    return rts